In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 21.3 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import numpy as np
import torch
import av
from collections import defaultdict
from tqdm import tqdm
from transformers import VivitImageProcessor, VivitForVideoClassification
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# def load_feature_dict(file_path):
#     with open(file_path, 'rb') as f:
#         return pickle.load(f)

# def compute_cosine_similarity(feature, feature_dict):
#     similarities = {}
#     for class_label, max_pooled_feature in feature_dict.items():
#         sim = cosine_similarity(feature, max_pooled_feature.reshape(1, -1))[0][0]
#         similarities[class_label] = sim
#     return similarities

# def classify_video(file_path, image_processor, model, device, feature_dict):
#     container = av.open(file_path)
#     indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
#     if indices is None:
#         return None
#     video = read_video_pyav(container=container, indices=indices)
#     inputs = image_processor(list(video), return_tensors="pt").to(device)

#     with torch.no_grad():
#         outputs = model(**inputs, output_hidden_states=True)
#         hidden_states = outputs.hidden_states
#         last_hidden = hidden_states[-1]

#     features = last_hidden.mean(dim=1).cpu().numpy()
#     similarities = compute_cosine_similarity(features, feature_dict)
#     return max(similarities, key=similarities.get)

In [ ]:
import av

In [ ]:
from transformers import pipeline
from transformers import VivitImageProcessor

image_processor = VivitImageProcessor.from_pretrained("mlnha/vivit-b-16x2-kinetics400-finetuned-original")

video_cls = pipeline(model="mlnha/vivit-b-16x2-kinetics400-finetuned-original")
video_cls.image_processor = image_processor

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pickle

pkl_file_path = '/content/drive/MyDrive/CV/code/feature_dictionary/pkl/O_max_pooled_features.pkl'

# Pickle 파일 불러오기
with open(pkl_file_path, 'rb') as f:
    word_dictionary = pickle.load(f)

print(word_dictionary)

{'병원': array([ 1.08824739e+01, -2.17481327e+00,  1.00501137e+01,  4.12488270e+00,
        4.29353428e+00,  3.39437199e+00,  3.90950847e+00,  8.16795731e+00,
       -1.14817655e+00,  8.40614510e+00,  6.41518593e+00,  5.85678625e+00,
        7.75690937e+00,  7.69828844e+00,  8.94604397e+00,  1.27656059e+01,
        4.78994465e+00,  1.30035038e+01,  1.24062233e+01,  1.16827002e+01,
        9.13094139e+00,  8.70944214e+00,  1.48215437e+01,  1.20297031e+01,
        1.43484316e+01,  3.63838315e+00,  4.64097977e+00,  5.20556116e+00,
        5.06730223e+00,  7.43098879e+00,  4.02325779e-01,  1.15507135e+01,
        3.44396925e+00,  1.00121346e+01,  5.11279106e-02,  2.27290416e+00,
        6.74322796e+00,  5.24984312e+00,  7.59857178e+00,  1.12835350e+01,
        9.56303596e+00,  5.37303257e+00,  8.48093224e+00,  1.02122736e+01,
        1.08031025e+01,  1.19872875e+01,  1.11193295e+01,  5.82423782e+00,
        4.86036968e+00,  1.21646042e+01,  5.48452282e+00,  8.28041363e+00,
        1.7651

In [ ]:
word_keys = list(word_dictionary.keys())
print("단어 사전:", word_keys)

단어 사전: ['병원', '붕대', '구급차', '의사', '골절', '쓰러지다', '가렵다', '배고프다', '친구', '다리']


In [ ]:
# 수어 영상이 있는 폴더 경로
video_folder_path = '/content/drive/MyDrive/CV/기컴비_텀프/data/train_dataset/original/train'

# 각 수어 영상에 대해 처리
for word_video_folder in os.listdir(video_folder_path):
  print(word_video_folder)

병원
붕대
구급차
의사
골절
쓰러지다
가렵다
배고프다
친구
다리


In [ ]:
device = torch.device("cuda")

In [ ]:
import os
from collections import defaultdict
import numpy as np
import av
import torch
from transformers import VivitImageProcessor, VivitForVideoClassification
from tqdm import tqdm
import cv2
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# accuracy
correct_predictions = 0
total_videos = 0

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    #########################################
    if converted_len >= seg_len:
        print(converted_len,">=",seg_len)
        return None
    #########################################
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def load_word_dictionary(pkl_file_path):
    with open(pkl_file_path, 'rb') as f:
        word_dictionary = pickle.load(f)
    return word_dictionary

def process_video_files(folder_paths):
    global correct_predictions, total_videos
    image_processor = video_cls.image_processor
    model = video_cls.model
    model.to(device)
    feature_dict = defaultdict(list)

    for folder_path in folder_paths:
        # 폴더명을 클래스 이름으로 사용
        for class_label in tqdm(os.listdir(folder_path)):
            class_path = os.path.join(folder_path, class_label)
            if not os.path.isdir(class_path):
                continue

            # 클래스 별 폴더 내 모든 파일을 탐색
            for filename in os.listdir(class_path):
                if filename.endswith(".mp4"):
                    file_path = os.path.join(class_path, filename)
                    container = av.open(file_path)

                    # 32 프레임 샘플링
                    indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
                    ###################
                    if indices is None:
                        print(filename)
                        continue
                    ###################
                    video = read_video_pyav(container=container, indices=indices)

                    # 비디오를 모델에 맞게 준비
                    # video_array = np.array(video)
                    #########################################################
                    inputs = image_processor(list(video), return_tensors="pt").to(device)
                    #########################################################

                    # 모델을 통한 전파
                    with torch.no_grad():
                        outputs = model(**inputs, output_hidden_states=True)
                        logits = outputs.logits
                        hidden_states = outputs.hidden_states
                        last_hidden = hidden_states[-1]

                    # 클래스별로 마지막 히든 레이어의 특징 저장
                    predictions = torch.argmax(logits, dim=-1)
                    for idx, prediction in enumerate(predictions):
                        last_hidden = last_hidden[idx].cpu().numpy()
                        # print(last_hidden.shape)
                        feature_dict[class_label].append(last_hidden[idx])

                    for pred_feature in feature_dict[class_label]:
                        similarities = {}
                        for word, word_feature in word_dictionary.items():
                            similarity = cosine_similarity([pred_feature], [word_feature])[0][0]
                            similarities[word] = similarity
                        most_similar_word = max(similarities, key=similarities.get)
                        print(f"단어(class_label): {class_label}, 예측(most_similar_word): {most_similar_word}")

                    total_videos += 1
                    if most_similar_word == class_label:
                        correct_predictions += 1
    return feature_dict


# 단어 사전
pkl_file_path = '/content/drive/MyDrive/CV/code/feature_dictionary/pkl/O_max_pooled_features.pkl'
word_dictionary = load_word_dictionary(pkl_file_path)

# 폴더 경로 설정 및 함수 호출
folder_paths = ['/content/drive/MyDrive/CV/기컴비_텀프/data/train_dataset/original/train', '/content/drive/MyDrive/CV/기컴비_텀프/data/train_dataset/original/val']
all_features = process_video_files(folder_paths)

accuracy = correct_predictions / total_videos
print(f"Accuracy: {accuracy}")


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(mos

 10%|█         | 1/10 [01:42<15:18, 102.06s/it]

단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 붕대
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 다리
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 의사
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 배고프다
단어(class_label): 병원, 예측(most_similar_word): 병원
단어(class_label): 병원, 예측(most_similar_wor

 20%|██        | 2/10 [03:17<13:03, 97.93s/it] 

...

단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프다
단어(class_label): 배고프다, 예측(most_similar_word): 배고프

 90%|█████████ | 9/10 [05:53<00:39, 39.96s/it]

단어(class_label): 구급차, 예측(most_similar_word): 골절
단어(class_label): 구급차, 예측(most_similar_word): 골절
단어(class_label): 구급차, 예측(most_similar_word): 붕대
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 의사
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 쓰러지다
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 쓰러지다
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(most_similar_word): 배고프다
단어(class_label): 구급차, 예측(

100%|██████████| 10/10 [06:31<00:00, 39.18s/it]

단어(class_label): 쓰러지다, 예측(most_similar_word): 쓰러지다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 쓰러지다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 구급차
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 배고프다
단어(class_label): 쓰러지다, 예측(most_similar_word): 쓰러지다
단어(class_label): 쓰러지다, 예측(most_similar_word): 다리
단어(class_label): 쓰러지다, 예측(most_similar_word): 다리
단어(class_label): 쓰러지다, 예측(most_similar_word): 쓰러지다
단어(class_label): 쓰러지다, 예측(most_similar_word): 쓰러지다
단어(cla